In [1]:
import gym
import torch
import torch.nn as nn
import numpy as np
from matplotlib import image
from matplotlib import pyplot
##import cPickle as pickle
from torch.distributions import Categorical
from itertools import count
from torch import nn
from torch.optim import Adam

from collections import deque
import random
import os

In [2]:
class PolicyNet(nn.Module):
    def __init__(self, input_shape, action_size):
        super(PolicyNet, self).__init__()       
       
        
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            #nn.BatchNorm2d(32, affine=False),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            #nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            #nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),
            #nn.Flatten()
        )
        
        conv_out_size = self._get_conv_out(input_shape)
        #2304
        #print(conv_out_size)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, action_size)
            
        )
        
    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))
        
        
    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1)
        #print(conv_out.size())
        #print(conv_out.size(0), np.prod(conv_out.size()[1:-1]))
        ##fc_in = torch.reshape(conv_out, (conv_out.size(0), np.prod(conv_out.size()[1:])))
        nn_out = self.fc(conv_out)
        ##return Categorical(torch.sigmoid(nn_out))
        
        ##return Categorical(torch.softmax(nn_out, dim=1))
        return torch.softmax(nn_out, dim=1)
    
                
    
class CriticNet(nn.Module):
    def __init__(self, input_shape):
        super(CriticNet, self).__init__()       
       
        
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            #nn.BatchNorm2d(32, affine=False),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            #nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            #nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),
            #nn.Flatten()
        )
        
        conv_out_size = self._get_conv_out(input_shape)
        #2304
        #print(conv_out_size)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
            
        )
        
    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))
        
        
    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1)
        #print(conv_out.size())
        #print(conv_out.size(0), np.prod(conv_out.size()[1:-1]))
        ##fc_in = torch.reshape(conv_out, (conv_out.size(0), np.prod(conv_out.size()[1:])))
        nn_out = self.fc(conv_out)
        ##return Categorical(torch.sigmoid(nn_out))
        
        return nn_out 

In [3]:
def preprocess(I):
  """ preprocess 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """

  if I is None:
     return torch.zeros(80,80)

  I             = I[35:195]    # crop
  I             = I[::2,::2,0] # downsample by factor of 2
  I[I == 144]   = 0            # erase background (background type 1)
  I[I == 109]   = 0            # erase background (background type 2)
  I[I != 0  ]   = 1            # everything else (paddles, ball) just set to 1
  ##return torch.from_numpy(I.astype(np.float32).ravel()).unsqueeze(0)
  return I

In [4]:
device = 'cuda'
#device = 'cpu'


In [5]:
import torch
import numpy as np

from collections import Counter
import time

import wrapper
import dqn_model

### Play the pong game with a trained dqn agent
#LOAD_PATH = '17061_pong_policy_net.pt' 
RENDER = True
FPS = 100

## for playing first we initialize the env
#env = gym.make('PongNoFrameskip-v4')

UP_ACTION = 2

DOWN_ACTION = 3


env = gym.make('Pong-v0')
##input_shape = env.observation_space.shape[0]
#action_size = env.action_space.n
action_size = 2

##print("Env reward threshold: {}".format(env.spec.reward_threshold))
reward_list = list()

##input_shape = 6400
## initialize a model
#policy_net = PolicyNet(input_shape, action_size).eval().to(device)
## load the trained modelinput_shape
#print(torch.loPolicyNetd(LOAD_PATH))

#policy_net.load_state_dict(torch.load(LOAD_PATH))


pnet = torch.load('6.0_A2C_pnet.pt')
pnet.eval()

cnet = torch.load('6.0_A2C_cnet.pt')
cnet.eval()


## get the initial state
state = env.reset()
prev_x = None

##action_prob = net(torch.FloatTensor(state).unsqueeze(0).to(device))
rewards = []
action_count = Counter()
## play the game
#for i in range(10):
while True:
    
    cur_x  = preprocess(state)
    #print("cur_x = ", cur_x.shape)
    state  = cur_x - prev_x if prev_x is not None else preprocess(prev_x)##np.zeros(input_shape)
    #print("state = ", state.shape)
    prev_x = cur_x

    
    
    ## get start time
    start_ts = time.time()
    ## if render the game
    if RENDER:
        env.render()
        
        
    state = state.reshape(1, 1, 80, 80)
        
    #env.render()
    ## take an action sampled from a categorical distribution given the state
    action = pnet(torch.FloatTensor(state).to(device)).max(1)[1].item()
    
    if action == 0:
            action = UP_ACTION
    else:
            action = DOWN_ACTION
    ##action_prob = pnet(torch.FloatTensor(state).to(device))
    ##action = action_prob.sample()

    #print(entropy)
    #value = cnet(torch.FloatTensor(state).to(device))
    #values.append(value[0])
    #print(action)
    next_state, reward, is_done, _ = env.step(action) # take a random action
    rewards.append(reward)

    ## current state is next state now
    state = next_state        
    
    ##state = torch.FloatTensor(new_state).unsqueeze(0)
    action_count[action] += 1

    if is_done:
        break

    ep_reward = sum(rewards)    
    if RENDER:
        delta = 1/FPS - (time.time() - start_ts)
        if delta > 0:
            time.sleep(delta)

print("Total Reward: {}".format(ep_reward))
print("Number of actions: {}".format(action_count))

##env.close()

Total Reward: -5.0
Number of actions: Counter({3: 6660, 2: 2874})
